In [ ]:
! sudo apt-get -qq install -y pciutils

In [ ]:
! curl https://ollama.ai/install.sh | sh

In [ ]:
! pip -q install dspy-ai

In [3]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

In [4]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
# Stop this cell after you see success!
! ollama run llama3:8b

In [6]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [21]:
from dspy import OllamaLocal
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

llama3_lm = OllamaLocal(model="llama3:8b",
                        model_type="text",
                        base_url="http://localhost:11434",
                        temperature=0.7,)

dspy.settings.configure(lm=llama3_lm)

# Load math questions from the GSM8K dataset
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 20390.42it/s]


In [23]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [24]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

100%|██████████| 10/10 [00:57<00:00,  5.79s/it]


In [25]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 1 / 10  (10.0): 100%|██████████| 10/10 [01:32<00:00,  9.21s/it]


10.0

In [26]:
llama3_lm.inspect_history(n=1)




Given the fields `question`, produce the fields `answer`.

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Answer: 24

Question: A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?
Answer: 12

Question: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?
Answer: 2

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?
Reasoning: Let's think step by step in order to Here are the fields `question`, `reasoning`, and `answer`: --- Question: Megan pays $16 fo

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nAnswer: 24\n\nQuestion: A third of the contestants at a singing competition are female, and the rest are male. If there are 18 contestants in total, how many of them are male?\nAnswer: 12\n\nQuestion: Nancy bought a pie sliced it into 8 pieces. She gave 1/2 to Joe and Darcy, and she gave 1/4 to Carl. How many slices were left?\nAnswer: 2\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: Megan pays $16 for a shirt that costs $22 before sales. What is the amount of the discount?\nReasoning: Let's think step by step in order to Here are the fields `question`, `reasoning`, and `answer`: ---